In [1]:
# 1.导出GeoIP2-City-Locations-en.csv中的非英文city_name部分

In [4]:
#!/usr/bin/env python 3.7
# -*- coding:utf-8 -*-
import re
import pandas as pd

# 判断字符串是否为{空、英文、非英文}
def isEnglish(s):
    if s=='nan':return 0
    else:
        ans = re.search(r"[a-zA-Z\']+$", s)
        return 1 if ans else 2

def city_location_ansys(data="../data/GeoIP2-City-CSV/GeoIP2-City-CSV_20190625/GeoIP2-City-Locations-en.csv"):
    data=pd.read_csv(data,encoding='utf-8')
    print('原始数据大小',data.shape)
    #判断是否为英文表达，{空值、英文、非英文(包括异常值)}依次对应{0,1,2}
    data['isEnglish']=data['city_name'].astype(str).apply(isEnglish)
    #输出{空值、英文、非英文(包括异常值)}信息
    num_0=list(data['isEnglish']).count(0)
    print("city_name为空的有",num_0)
    num_1=list(data['isEnglish']).count(1)
    print("city_name为英文的有",num_1)
    num_2=list(data['isEnglish']).count(2)
    print("city_name为非英文(包括异常字符)的有",num_2)
    #将非英文部分提取出，以用于数据爬取转换：
    abnormal=data[data['isEnglish'] ==2]
    print(abnormal.shape)
    abnormal.to_csv('../output/GeoIP2-City-Locations-abnormal.csv')

if __name__=='__main__':
    city_location_ansys()


原始数据大小 (145718, 14)
city_name为空的有 1173
city_name为英文的有 144115
city_name为非英文(包括异常字符)的有 430
(430, 15)


In [5]:
# 2.批量地理逆编码(偶尔出现raise GeocoderTimedOut('Service timed out')是正常现象，说明访问过于频繁导致服务器无响应，程序将会自动重发请求)

In [4]:
#!/usr/bin/env python 3.7
# -*- coding:utf-8 -*-
import geopy.geocoders  # doc：https://geopy.readthedocs.io/en/stable/
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3, Yandex, Photon
from geopy.extra.rate_limiter import RateLimiter  # for dataframe quering
import pandas as pd
from tqdm import tqdm
import re
import functools

UA = "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0"

# default param
geopy.geocoders.options.default_user_agent = UA
geopy.geocoders.options.default_timeout = 7  # 抛出异常前等待地理编码服务响应的时间(s)
# set Geo-coder
# geolocator=GoogleV3(user_agent=UA)#自2018/07起，Google要求每个请求都需要API秘钥
geolocator=Nominatim(user_agent=UA)
# geolocator = Yandex(user_agent=UA, lang='en')
# geolocator=Photon()#此平台无需秘钥,较稳定

# 给定单个维度经度lat-lon，获取其location
def geo_reverse_coding(latitude, longitude, UA=UA):
    address = str(latitude) + ',' + str(longitude)
    location = geolocator.reverse(address, language='en')
    print('Lat&Lon:', address)
    print('location:', location.address)
    print('location.raw', location.raw)
    return location

#获取geoname_id和lat&lon的索引关系
def get_geoID():
    id_v4=pd.read_csv("../data/GeoIP2-City-CSV/GeoIP2-City-CSV_20190625/GeoIP2-City-Blocks-IPv4.csv",encoding='utf-8')
    id_v6=pd.read_csv("../data/GeoIP2-City-CSV/GeoIP2-City-CSV_20190625/GeoIP2-City-Blocks-IPv6.csv",encoding='utf-8')
    print("IPv4_shape=",id_v4.shape,"IPv6_shape=",id_v6.shape)
    #为降低数据量，仅保留geoname_id、latitude、longitude三个特征列来作为索引依据
    bad_cols=['network','registered_country_geoname_id','represented_country_geoname_id','is_anonymous_proxy',
              'is_satellite_provider','postal_code','accuracy_radius']
    for df in [id_v4,id_v6]:
        df.drop(bad_cols,axis=1,inplace=True)
    print('remaining_columns:',id_v4.columns)
    geoID=pd.concat([id_v4,id_v6],axis=0,ignore_index=True)
    del id_v4
    del id_v6
    print("get geoID!")
    print("geoID shape:", geoID.shape)
    #删除geoname_id中重复的行
    geoID.drop_duplicates(subset=['geoname_id',],keep='first',inplace=True)
    print("geoID shape after duplicates:",geoID.shape)
    return geoID

#将lat&lon（纬度和经度）merge到data中
def merge_geoID_data(geoID,data="../output/GeoIP2-City-Locations-abnormal.csv"):
    data=pd.read_csv(data,encoding='utf-8')
    print('merging...')
    res=pd.merge(data,geoID,on='geoname_id')
    res.to_csv('../output/GeoIP2-City-Locations-abnormal-vsLatLot.csv')
    del data
    del geoID
    return res

#批量逆编码
def batch_geo_reverse_coding(data):
    #使用偏函数对默认参数进行修改
    print('ds',data.shape)
    georeverse = functools.partial(geolocator.reverse, language='en')
    # 自动添加请求延迟，减少负载，且能够重试失败的请求并过滤各行的错误提示
    location = RateLimiter(georeverse, min_delay_seconds=1, )
    data['latitude'] = data['latitude'].astype(str)
    data['longitude'] = data['longitude'].astype(str)
    data['LatLon'] = data['latitude'].str.cat(data['longitude'], sep=',')
    tqdm.pandas()# 调用tqdm进度条库显示进度,需要改用progress_apply
    data['location_raw'] = data['LatLon'].progress_apply(location)
    # data['city']=data['location_raw'].apply(lambda x:re.findall(r"[(](.*?)[(]",str(x)))
    data.to_csv('../output/GeoIP2-City-Locations-abnormal-resRaw.csv')
    return data


if __name__ == '__main__':
    #单条转换
    #geo_reverse_coding(latitude='39.956981',longitude='116.375523')#latitude='13.6967',longitude='44.7308'
    #批量转换
    geoID=get_geoID()#获取IPv4、IPv6的geoname_id -- Lat&Lon关系表
    data=merge_geoID_data(geoID,)#将表merge到需要逆编码的data中，使data具有lat&lon列
    batch_geo_reverse_coding(data)#批量进行逆编码

IPv4_shape= (8703271, 10) IPv6_shape= (1011870, 10)
remaining_columns: Index(['geoname_id', 'latitude', 'longitude'], dtype='object')
get geoID!
geoID shape: (9715141, 3)
geoID shape after duplicates: (145716, 3)
merging...
ds (430, 18)




  0%|                                                                                          | 0/430 [00:00<?, ?it/s]

  0%|▍                                                                                 | 2/430 [00:01<04:47,  1.49it/s]

  1%|▌                                                                                 | 3/430 [00:06<15:07,  2.12s/it]

  1%|▊                                                                                 | 4/430 [00:13<24:02,  3.39s/it]

  1%|▉                                                                                 | 5/430 [00:15<22:17,  3.15s/it]

  1%|█▏                                                                                | 6/430 [00:19<24:15,  3.43s/it]

  2%|█▎                                                                                | 7/430 [00:23<24:22,  3.46s/it]

  2%|█▌                                                                                | 8/430 [00:25<22:29,  3.20s/it]

  2%|█▋                       



  8%|██████▌                                                                          | 35/430 [02:19<46:32,  7.07s/it]

  8%|██████▊                                                                          | 36/430 [02:24<42:24,  6.46s/it]

  9%|██████▉                                                                          | 37/430 [02:27<33:59,  5.19s/it]

  9%|███████▏                                                                         | 38/430 [02:33<36:26,  5.58s/it]

  9%|███████▎                                                                         | 39/430 [02:36<29:59,  4.60s/it]

  9%|███████▌                                                                         | 40/430 [02:40<30:03,  4.62s/it]

 10%|███████▋                                                                         | 41/430 [02:43<25:39,  3.96s/it]

 10%|███████▉                                                                         | 42/430 [02:45<22:26,  3.47s/it]

 10%|████████                 

 29%|██████████████████████▉                                                         | 123/430 [07:27<14:15,  2.79s/it]

 29%|███████████████████████                                                         | 124/430 [07:30<14:52,  2.92s/it]

 29%|███████████████████████▎                                                        | 125/430 [07:32<13:41,  2.69s/it]

 29%|███████████████████████▍                                                        | 126/430 [07:34<12:58,  2.56s/it]

 30%|███████████████████████▋                                                        | 127/430 [07:37<12:24,  2.46s/it]

 30%|███████████████████████▊                                                        | 128/430 [07:39<12:28,  2.48s/it]

 30%|████████████████████████                                                        | 129/430 [07:41<11:54,  2.37s/it]

 30%|████████████████████████▏                                                       | 130/430 [07:44<11:59,  2.40s/it]

 30%|████████████████████████▎  

 54%|███████████████████████████████████████████▌                                    | 234/430 [13:06<09:58,  3.05s/it]

 55%|███████████████████████████████████████████▋                                    | 235/430 [13:09<09:35,  2.95s/it]

 55%|███████████████████████████████████████████▉                                    | 236/430 [13:12<09:26,  2.92s/it]

 55%|████████████████████████████████████████████                                    | 237/430 [13:16<11:06,  3.46s/it]

 55%|████████████████████████████████████████████▎                                   | 238/430 [13:20<11:19,  3.54s/it]

 56%|████████████████████████████████████████████▍                                   | 239/430 [13:23<10:18,  3.24s/it]

 56%|████████████████████████████████████████████▋                                   | 240/430 [13:25<09:27,  2.99s/it]

 56%|████████████████████████████████████████████▊                                   | 241/430 [13:28<08:53,  2.82s/it]

 56%|███████████████████████████

 86%|████████████████████████████████████████████████████████████████████▍           | 368/430 [19:39<03:50,  3.71s/it]

 86%|████████████████████████████████████████████████████████████████████▋           | 369/430 [19:41<03:21,  3.30s/it]

 86%|████████████████████████████████████████████████████████████████████▊           | 370/430 [19:44<03:03,  3.06s/it]

 86%|█████████████████████████████████████████████████████████████████████           | 371/430 [19:46<02:47,  2.84s/it]

 87%|█████████████████████████████████████████████████████████████████████▏          | 372/430 [19:52<03:29,  3.61s/it]

 87%|█████████████████████████████████████████████████████████████████████▍          | 373/430 [19:54<03:02,  3.20s/it]

 87%|█████████████████████████████████████████████████████████████████████▌          | 374/430 [19:58<03:09,  3.38s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('7.9667,-80.4333',), **{}).
Traceback (most recent call last):
  File "D:\01software\Anac

 95%|███████████████████████████████████████████████████████████████████████████▉    | 408/430 [22:53<01:55,  5.25s/it]

 95%|████████████████████████████████████████████████████████████████████████████    | 409/430 [22:55<01:31,  4.37s/it]

 95%|████████████████████████████████████████████████████████████████████████████▎   | 410/430 [22:58<01:16,  3.81s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('25.787,-100.3368',), **{}).
Traceback (most recent call last):
  File "D:\01software\Anaconda3\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "D:\01software\Anaconda3\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "D:\01software\Anaconda3\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "D:\01software\Anaconda3\lib\http\client.py", line 1224, in endheaders
    self._send_ou

In [5]:
# 3.调整以上获得的临时文件resRaw，使其与原始数据GeoIP2-City-Locations-en.csv的格式保持一致，并在output文件夹输出最终结果...res.csv

In [9]:
#删除临时列，将dataframe调整至标准状态
def get_finall_result(data='../output/GeoIP2-City-Locations-abnormal-resRaw.csv'):
    data=pd.read_csv(data,encoding='utf-8')
    print(data.columns)
    good_cols=['geoname_id','locale_code','continent_code','continent_name','country_iso_code','country_name',
               'subdivision_1_iso_code','subdivision_1_name','subdivision_2_iso_code','subdivision_2_name','city_name',
               'metro_code','time_zone','is_in_european_union',]+['location_raw',]#需要保留的列
    for col in data.columns:
        if col not in good_cols:
            data.drop(col,axis=1,inplace=True)
    #print(data.shape,data.columns)
    #将city_name覆盖重写
    data['location_raw']=data['location_raw'].astype(str)
    data['city_name']=data['location_raw'].str.split(',').apply(lambda x:x[0][10:])#[10:]
    # data['city'] = data['location_raw'].str.split(',')# [10:]
    # data['city']=data['location_raw'].str.split(',')[0][10:]
    data.drop('location_raw',axis=1,inplace=True)
    data.to_csv('../output/GeoIP2-City-Locations-abnormal-res.csv')
    print("finish！最终文件位于‘../output/GeoIP2-City-Locations-abnormal-res.csv’")

if __name__ == '__main__':
    get_finall_result()#调整列使其与原始数据data格式一致

Index(['Unnamed: 0', 'Unnamed: 0.1', 'geoname_id', 'locale_code',
       'continent_code', 'continent_name', 'country_iso_code', 'country_name',
       'subdivision_1_iso_code', 'subdivision_1_name',
       'subdivision_2_iso_code', 'subdivision_2_name', 'city_name',
       'metro_code', 'time_zone', 'is_in_european_union', 'isEnglish',
       'latitude', 'longitude', 'LatLon', 'location_raw'],
      dtype='object')
finish！最终文件位于‘../output/GeoIP2-City-Locations-abnormal-res.csv’
